In [1]:
import tensorflow as tf
import os

In [2]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.compat.v1.random.set_random_seed(1)

import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame
from math import sqrt
import os
from sklearn.model_selection import KFold
# import shutil
import json
import re

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import *

In [3]:
model = tf.keras.models.load_model("D:\saving Mydoc\Kuliah\PKL\File saya\Weighted\code_keras\saved_models_multi\Transformer_B18_False_head1\Transformer_h32_bs50_ep100_w5_Adam_lr0.001_k1\saved_model_and_weights")

In [4]:
def preprocess(dataset):
    scalers = {}
    for i in range(dataset.shape[1]):
      scalers[i] = MinMaxScaler(feature_range=(0, 1))
      dataset[:,i,:] = scalers[i].fit_transform(dataset[:,i,:])
    return dataset, scalers

In [5]:
def extract_VIT_capacity(x_datasets, y_datasets, seq_len, hop, sample):
    x = [] # VITC = inputs voltage, current, temperature (in vector) + capacity (in scalar)
    y = [] # target capacity (in scalar)
    z = [] # cycle index
    SS = [] # scaler
    VITC = [] # temporary input

    for x_data, y_data in zip(x_datasets, y_datasets):
        # Load VIT from charging profile
        x_df = read_csv(x_data).dropna()
        x_df = x_df[['cycle','voltage_battery','current_battery','temp_battery']]
        x_df['cycle'] = x_df['cycle']+1
        x_len = len(x_df.cycle.unique()) #- seq_len

        # Load capacity from discharging profile
        y_df = read_csv(y_data).dropna()
        y_df['cycle_idx'] = y_df.index+1
        y_df = y_df[['capacity', 'cycle_idx']]
        y_df = y_df.values # Convert pandas dataframe to numpy array
        y_df = y_df.astype('float32') # Convert values to float
        y_len = len(y_df) #- seq_len

        data_len = np.int32(np.floor((y_len - seq_len-1)/hop)) + 1
        for i in range(y_len):
            cy = x_df.cycle.unique()[i]
            df = x_df.loc[x_df['cycle']==cy]
            # Voltage measured
            le = len(df['voltage_battery']) % sample
            vTemp = df['voltage_battery'].to_numpy()
            if le != 0:
                vTemp = vTemp[0:-le]
            vTemp = np.reshape(vTemp, (len(vTemp)//sample,-1)) #, order="F")
            vTemp = vTemp.mean(axis=0)
            # Current measured
            iTemp = df['current_battery'].to_numpy()
            if le != 0:
                iTemp = iTemp[0:-le]
            iTemp = np.reshape(iTemp, (len(iTemp)//sample,-1)) #, order="F")
            iTemp = iTemp.mean(axis=0)
            # Temperature measured
            tTemp = df['temp_battery'].to_numpy()
            if le != 0:
                tTemp = tTemp[0:-le]
            tTemp = np.reshape(tTemp, (len(tTemp)//sample,-1)) #, order="F")
            tTemp = tTemp.mean(axis=0)
            # Capacity measured
            cap = np.array([y_df[i, 0]])
            # Combined
            VITC_inp = np.concatenate((vTemp, iTemp, tTemp, cap))
            VITC.append(VITC_inp)

        # Normalize using MinMaxScaler
        df_VITC = DataFrame(VITC).values
        scaled_x, scaler = preprocess(df_VITC[:, :, np.newaxis])
        scaled_x = scaled_x.astype('float32')[:, :, 0] # Convert values to float

        # Create input data
        for i in range(data_len):
            x.append(scaled_x[(hop*i):(hop*i+seq_len), :])
            y.append(scaled_x[hop*i+seq_len, -1])
            # z.append(y_df[hop*i+seq_len, 1])
        SS.append(scaler)
        # import pdb; pdb.set_trace()
    return np.array(x), np.array(y)[:, np.newaxis], SS

In [10]:
pth = "Data/NASA/"
train_x_files = [os.path.join(pth,'charge/train',f) for f in os.listdir(os.path.join(pth, 'charge/train'))]
train_x_files.sort(key=lambda f: int(re.sub('\D', '', f)))
train_y_files = [os.path.join(pth,'discharge/train',f) for f in os.listdir(os.path.join(pth, 'discharge/train'))]
train_y_files.sort(key=lambda f: int(re.sub('\D', '', f)))
test_x_data = [os.path.join(pth,'charge/test',f) for f in os.listdir(os.path.join(pth, 'charge/test'))]
test_y_data = [os.path.join(pth,'discharge/test',f) for f in os.listdir(os.path.join(pth, 'discharge/test'))]
print("train X:", train_x_files)
print("train Y:", train_y_files)

train X: ['Data/NASA/charge/train\\B0005_charge.csv', 'Data/NASA/charge/train\\B0007_charge.csv', 'Data/NASA/charge/train\\B0018_charge.csv']
train Y: ['Data/NASA/discharge/train\\B0005_discharge.csv', 'Data/NASA/discharge/train\\B0007_discharge.csv', 'Data/NASA/discharge/train\\B0018_discharge.csv']


In [11]:
test_x_data

['Data/NASA/charge/test\\B0006_charge.csv']

In [24]:
testX, testY, SS_tt = extract_VIT_capacity(test_x_data, test_y_data, 5, 1, 10)

In [40]:
testX.shape

(127, 5, 31)

In [41]:
testY.shape

(127, 1)

In [25]:
testPredict = model.predict(testX)

In [26]:
results = model.evaluate(testX, testY)
test_loss.append(results)
print('Test loss:', results)

4/4 [==============================] - 0s 27ms/step - loss: 0.0222
Test loss: 0.02220015600323677


In [28]:
inv_testY = SS_tt[0][30].inverse_transform(testY)
inv_testPredict = SS_tt[0][30].inverse_transform(testPredict)

In [29]:
test_mae = mean_absolute_error(inv_testY, inv_testPredict)
test_mse = mean_squared_error(inv_testY, inv_testPredict)
test_mape = mean_absolute_percentage_error(inv_testY, inv_testPredict)
test_rmse = np.sqrt(mean_squared_error(inv_testY, inv_testPredict))
print('\nTest Mean Absolute Error: %f MAE' % test_mae)
print('Test Mean Square Error: %f MSE' % test_mse)
print('Test Mean Absolute Percentage Error: %f MAPE' %test_mape)
print('Test Root Mean Squared Error: %f RMSE' % test_rmse)
# import pdb; pdb.set_trace()


Test Mean Absolute Error: 0.067614 MAE
Test Mean Square Error: 0.005864 MSE
Test Mean Absolute Percentage Error: 0.045026 MAPE
Test Root Mean Squared Error: 0.076578 RMSE


In [12]:
def plot_pred(predict, true, save_dir, model_dir, name):    # Plot test prediction
    predict = predict.reshape(predict.shape[0])
    true = true.reshape(true.shape[0])
    fig=plt.figure(figsize=(12, 4),dpi=150)
    plt.plot(predict,label='Prediction')
    plt.plot(true,label='True')
    plt.xlabel('Number of Cycle',fontsize=13)
    plt.ylabel('DIscharge Capacity (Ah)',fontsize=13)
    # plt.title(str(model_name.__name__)+' Prediction of Discharge Capacity of Test Data (B0005)',fontsize=14)
    plt.legend(loc='upper right',fontsize=12)
#     plt.savefig(os.path.join(save_dir, model_dir, name+'.png'))

In [13]:
plot_pred(inv_testPredict, inv_testY, None, None,name='test_pred')

NameError: name 'inv_testPredict' is not defined